In [2]:
%pip install monai

# Imports

In [3]:
import os
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import monai
from monai.data import CacheDataset, MetaTensor
from monai.transforms import Compose, SpatialPadd, CenterSpatialCropd, ScaleIntensityd, EnsureChannelFirstd, Transposed

# Function to load and preprocess Nifti images

In [9]:
def load_images(directory_pelvis: str):
    """
    Load nifti images and preprocess them for each patient pair MRI-CT.

    Args:
        directory_pelvis (str): directory containing each patient's folders.
    
    Return:
        list: A list of dictionaries, each containing 'mri' and 'ct' images as numpy arrays.
    """
    dataset = []
    
    for folder_patient in os.listdir(directory_pelvis)[:5]:
        patient_path = os.path.join(directory_pelvis, folder_patient)

        # Load MRI and CT images for each patient
        mri_path = os.path.join(patient_path, "mr.nii.gz")
        ct_path = os.path.join(patient_path, "ct.nii.gz")

        if os.path.exists(mri_path) and os.path.exists(ct_path):
            mri_image = nib.load(mri_path).get_fdata()  # Load as numpy array
            ct_image = nib.load(ct_path).get_fdata()    # Load as numpy array

            
            
            # Append the MRI-CT pair and label to the dataset
            dataset.append({"mr": mri_image, "ct": ct_image})

    return dataset

dataset = load_images("C:\\Users\\catar\\OneDrive - Universidade de Coimbra\\Ambiente de Trabalho\\Master Thesis\\Dataset SynthRAD\\Task1\\pelvis")

In [ ]:
def standardize_mri_ct_pairs(dataset):
    """
    Standardizes a dataset of MRI-CT pairs by calculating a consistent shape across all pairs.
    
    Args:
        dataset: List of dataset items, each containing "mr" and "ct" keys with NIfTI images.

    Returns:
        CacheDataset: A standardized dataset with MRI-CT pairs of consistent shape and intensity.
    """
    
    # Step 1: Calculate the average target shape (new_size) based on all MRI-CT pairs (each pair has the same shape) 
    all_shapes = []
    for element in dataset:
        all_shapes.append(element['mr'].shape[:3])

    new_size = (2 ** np.round(np.log2(np.array(all_shapes).mean(0)))).astype(int)
    print(f"Calculated target shape (new_size): {new_size}")

    # Step 2: Define transformations to apply to each MRI-CT pair
    transform = Compose(
        [
            # Move channel to the first dimension if needed
            EnsureChannelFirstd(keys=["mr", "ct"], channel_dim=0),
            # Normalize intensities for MRI and CT images
            ScaleIntensityd(keys=["mr", "ct"]),
            # Pad or crop to the calculated target shape
            SpatialPadd(keys=["mr","ct"], spatial_size=tuple(new_size)),
            CenterSpatialCropd(keys=["mr", "ct"], roi_size=tuple(new_size))
        ]
    )
    
    # Step 3: Apply transformations and cache the dataset
    standardized_dataset = monai.data.CacheDataset(data=dataset, transform=transform, cache_rate=1.0)
    
    return standardized_dataset


In [17]:
standardized_dataset = standardize_mri_ct_pairs(dataset)

Calculated target shape (new_size): [512 256 128]


RuntimeError: applying transform <monai.transforms.croppad.array.SpatialPad object at 0x0000025EDF37AED0>

# GAN Model Setup
https://medium.com/@Avizyt/understanding-generative-neural-networks-with-pytorch-a-step-by-step-guide-164a314062e6

In [7]:
# Define the Generator network
class Generator(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, output_dim),
            nn.Tanh()  # Using Tanh activation for the final output to ensure values are in the range [-1, 1]
        )

    def forward(self, x):
        return self.model(x)

# Define the Discriminator network
class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 128),
            nn.LeakyReLU(0.2),
            nn.Linear(128, 1),
            nn.Sigmoid()  # Using Sigmoid activation to get a probability value between 0 and 1
        )

    def forward(self, x):
        return self.model(x)

# Instances of the generator and discriminator

In [9]:
latent_dim = 100
input_dim = 100
output_dim = 50*50
generator = Generator(latent_dim, output_dim) # Define the output shape for your generator
discriminator = Discriminator(input_dim) # Define the input shape for your discriminator

# Loss function and optimizers for generator and discriminator

In [10]:
criterion = nn.BCELoss()
optimizer_G = torch.optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

# Training the GAN

In [14]:
# Training loop
num_epochs = 10000
batch_size = 16

for epoch in range(num_epochs):
    for i in range(0, medical_images.shape[0], batch_size):
        # Sample a batch of real medical images
        real_images = medical_images[i:i + batch_size]

        # Generate random noise as input for the generator
        noise = torch.randn(batch_size, latent_dim)

        # Generate fake medical images using the generator
        fake_images = generator(noise)

        # Train the discriminator on real images
        optimizer_D.zero_grad()
        real_labels = torch.ones(batch_size, 1)
        d_loss_real = criterion(discriminator(real_images), real_labels)
        d_loss_real.backward()

        # Train the discriminator on fake images
        fake_labels = torch.zeros(batch_size, 1)
        d_loss_fake = criterion(discriminator(fake_images.detach()), fake_labels)
        d_loss_fake.backward()
        optimizer_D.step()

        # Calculate discriminator loss
        d_loss = d_loss_real + d_loss_fake

        # Train the generator
        optimizer_G.zero_grad()
        g_loss = criterion(discriminator(fake_images), real_labels)
        g_loss.backward()
        optimizer_G.step()

        # Print progress
        print(f"Epoch [{epoch}/{num_epochs}], Batch [{i}/{medical_images.shape[0]}], D Loss: {d_loss.item()}, G Loss: {g_loss.item()}")

AttributeError: 'list' object has no attribute 'shape'